# Notebook
This is just a way of making it easier to go through the audio features, and share any work that I've been doing. This is done by Nicolas

This will probably be a bit messy, but it exists and will be commented and that's what matters

In [1]:
%pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'E:\Projects\StefKaptein\social-signal-processing-project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [40]:
import pandas as pd

from text_features import get_sentence_similarity

master_df = pd.read_csv("all_datasets.csv", sep=';')

master_df.head()

,segID,StartTimeA,EndTimeA,StartTimeB,EndTimeB,f0_means,f0_stds,pause,speakerChange,similarity,boundary
0,Bed002.segment.1,5.790,6.200,6.393,7.043,"['1.64607', '0.59155']","['4.02422466667', '50.9371406173']",0.193,True,[[0.80745196]],0.0
1,Bed002.segment.2,6.393,7.043,10.383,12.373,['1.37410'],['11.8302375346'],3.340,False,[[0.54926735]],0.0
2,Bed002.segment.3,10.383,12.373,12.573,13.723,"['0.55294', '1.02994', '0.90475', '0.59609', '...","['45.4319839339', '24.0814492188', '35.0004665...",0.200,False,[[0.7473101]],0.0
3,Bed002.segment.4,12.573,13.723,12.822,14.542,"['0.00000', '1.10862', '1.07281', '0.87544']","['0.0', '2.2370073', '0.557957183673', '40.790...",-0.901,True,[[0.4641241]],0.0
4,Bed002.segment.5,12.822,14.542,14.655,16.180,"['0.35041', '1.03134', '0.33097', '0.85372']","['37.9096119852', '42.0439837127', '35.8062448...",0.113,True,[[0.44520015]],0.0


In [56]:
# So, for starters I'll just do a pretty crappy classifier. Reading all of th datasets doesn't make sense, as it makes it confusing dealing with when one dataset ends and the next one starts...

# So... regardless of what features we use, the classifier should use something as follows:
#. Transform each pair of rows into one row, to make it easier to feed
#. The classifier will be true if the previous is 1 and the next is 0. Because that's how it was formatted

def transform_rows(dataframe: pd.DataFrame, features: list):
    """Receives the original dataframe, loops through all of the rows and transforms them to
    be valid for a generic classifier.

    Method assumes that the boundary column always exists. When the value is a 1, that means
    it's the end of the topic. Which is the important part for the topic segmentation.

    Method automatically adds boundary to the list of features, because it should always be used

    :param dataframe: Row dataframe that contains all of the information that will be processed
    :param features: The columns that will be kept of the dataframe. Assumed to be list like that contains
    the titles of any features that are to be kept. In case further transformation is required,
    it will be done externally. Boundary is not included in this list

    :returns Transformed dataframe that already contains the relevant columns. Each feature can
    """

    # I will add boundary to the list afterwards
    features.append('boundary')

    filtered_df = dataframe[features]

    temp_df = pd.concat([filtered_df.add_suffix('1'), filtered_df.shift(-1).add_suffix('2')], axis=1)

    # Drop the last entry, as it'll be comparing a row with nothing. Which isn't particularly helpful
    temp_df.drop(temp_df.tail(1).index,inplace=True)

    # Now the last step is just to do a check for whether it is a boundary set of rows, or not.
    #. And to remove the boundary column as well.
    boundary = (temp_df['boundary1'] == 1.0) & (temp_df['boundary2'] == 0.0)

    temp_df.drop(['boundary1', 'boundary2'], axis=1, inplace=True)
    temp_df['boundary'] = boundary

    return temp_df

In [58]:
to_transform_df = pd.read_csv('../results_merged/Bed002.csv', sep=';')
temp_df = transform_rows(to_transform_df, ['pause', 'speakerChange'])
temp_df.head()

,pause1,speakerChange1,pause2,speakerChange2,boundary
0,0.193,True,3.340,False,False
1,3.340,False,0.200,False,False
2,0.200,False,-0.901,True,False
3,-0.901,True,0.113,True,False
4,0.113,True,0.986,True,False


In [59]:
# With the way the pause and the speaker change work, it just makes sense to drop the other two elements
df = temp_df.drop(['pause2', 'speakerChange2'], axis=1)

# This is gonna be a pretty crap classifier...

13